[requests-html快速入门](https://segmentfault.com/a/1190000015641160)

https://stackoverflow.com/questions/51643629/querying-div-class-name-in-python

# 抓取全部商品的連結

In [1]:
import requests
from requests_html import HTML
import pandas as pd
import re

In [2]:
url = 'https://www.pcone.com.tw/product/'
# 男生服飾
info = '327' 

# 加入使用者資訊(如使用什麼瀏覽器、作業系統...等資訊)模擬真實瀏覽網頁的情況
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

> 選擇男生服飾這個類別是因為商品的數量較少，一方面讓我們可以比較快速地抓資料，另方面也比較不會造成店家的困擾。

透過檢查的功能，可以看到商品的連結都藏在ul a 的下面，因此我們可以透過以下方式提取這些連結

In [3]:
resp = requests.get(url + info, headers=headers)  
html = HTML(html=resp.text)
a = html.find('a.product-list-item')
a

[<Element 'a' class=('product-list-item',) data-app='go-product' href='/product/info/190117048588?t=home_hotitem#ref=d_index_hot_1' target='_blank'>,
 <Element 'a' class=('product-list-item',) data-app='go-product' href='/product/info/180823036685?t=home_hotitem#ref=d_index_hot_2' target='_blank'>,
 <Element 'a' class=('product-list-item',) data-app='go-product' href='/product/info/181212921322?t=home_hotitem#ref=d_index_hot_3' target='_blank'>,
 <Element 'a' class=('product-list-item',) data-app='go-product' href='/product/info/180831227042?t=home_hotitem#ref=d_index_hot_4' target='_blank'>,
 <Element 'a' class=('product-list-item',) data-app='go-product' href='/product/info/181011285850?t=home_hotitem#ref=d_index_hot_5' target='_blank'>,
 <Element 'a' class=('product-list-item',) data-app='go-product' href='/product/info/190118229702?t=home_hotitem#ref=d_index_hot_6' target='_blank'>,
 <Element 'a' class=('product-list-item',) data-app='go-product' href='/product/info/170809105334?t=

OK! 在這裡我們確認可以把商品找出來囉!

撰寫簡單的函數，當我們輸入商品的類型時，自動幫我們帶入該類型下的所有商品

# 抓取個別商品資訊
接下來要繼續模擬，如果我進到商品的網頁之後，如何能把商品的名稱/銷售量/價格/原價等內容抓下來了？

In [4]:
# 這裡加入這個網址的畫面
'https://www.pcone.com.tw/' + a[0].attrs['href']

'https://www.pcone.com.tw//product/info/190117048588?t=home_hotitem#ref=d_index_hot_1'

In [5]:
resp = requests.get('https://www.pcone.com.tw/' + a[0].attrs['href'], headers=headers)
html = HTML(html=resp.text)

In [6]:
# 店家名稱
html.find('a.store-name',first = True).text

'【好康市集】'

In [7]:
# 店家商品數量
html.find('div.store-val',first = False)[0].attrs['data-val']

'28'

In [8]:
# 店家評價
html.find('div.store-val',first = False)[1].attrs['data-val']

'4.6'

In [9]:
# 店家出貨天數
html.find('div.store-val',first = False)[2].attrs['data-val']

'0.9'

In [10]:
# 店家回覆率
html.find('div.store-val',first = False)[3].attrs['data-val']

'94%'

In [11]:
# 產品名稱
html.find('h1.product-name',first = True).text

'【瑞典】旅行折疊電熱水壺'

In [12]:
# 特價
html.find('span.bind-lowest-price.discount',first = True).text

'$1280'

In [13]:
# 原價
html.find('span.original',first = True).text

'$1699'

In [14]:
# 折數
html.find('span.bind-discount-number.discount-number',first = True).text

'7.5折'

In [15]:
# 商品評分
html.find('span.count > span',first = False)[0].text

'3.5'

In [16]:
# 評價人數
html.find('span.count > span',first = False)[1].text

'2'

In [17]:
# 收藏人數
html.find('div.count',first = False)[0].text

'64'

In [18]:
# 提問人數
html.find('div.count',first = False)[1].text

'2'

In [19]:
# 商品分類
html.find('div.breadcrumbs-set',first = True).text

'戶外休閒 旅行用品 旅用配件'

In [20]:
# 商品標籤
html.find('div.tags',first = True).text

'#攜帶水壺 #電熱水壺 #旅行配件 #旅行水壺 #摺疊水壺 #收納水壺'

In [21]:
# 連結
'https://www.pcone.com.tw' + a[0].attrs['href']

'https://www.pcone.com.tw/product/info/190117048588?t=home_hotitem#ref=d_index_hot_1'

> 客戶留言也是同樣的操作邏輯，在此不多練習

在這我們先嘗試將以上內容定義成一個函數，只要輸入包含有商品ID的字串，就幫我們輸出商品的資訊

# 串接以上兩個語法
簡單回顧一下，我們首先先寫了一個語法，讓我們只要輸入類型的編號，就會回傳該類型下共有那些商品連結。
並且在第二部分則只要輸入包含商品ID的字串，就幫我們回傳商品的各種資訊。
因此在這裡我們透過迴圈，把第一個函數的輸出結果逐一爬商品屬性的話，就能夠!!!


In [22]:
def ProdList(info):
    resp = requests.get(url + str(info), headers=headers)
    html = HTML(html=resp.text)
    return(html.find('a.product-list-item'))

In [23]:
def Crawl_SongGuo(info):
    resp = requests.get('https://www.pcone.com.tw/product/info/' + re.search(r'\d{12}',str(info)).group(), headers=headers)
    html = HTML(html=resp.text)
    print(re.search(r'\d{12}',str(info)).group())
    return(pd.DataFrame(
            data=[{
                '店家名稱':html.find('a.store-name',first = True).text,
                '店家商品數量':html.find('div.store-val',first = False)[0].attrs['data-val'],
                '店家評價':html.find('div.store-val',first = False)[1].attrs['data-val'],
                '店家出貨天數':html.find('div.store-val',first = False)[2].attrs['data-val'],
                '店家回覆率':html.find('div.store-val',first = False)[3].attrs['data-val'],
                '產品名稱':html.find('h1.product-name',first = True).text,
                '特價':html.find('span.bind-lowest-price.discount',first = True).text,
                '原價':html.find('span.original',first = True).text,
                '折數':html.find('span.bind-discount-number.discount-number',first = True).text,
                '商品評分':html.find('span.count > span',first = False)[0].text,
                '評價人數':html.find('span.count > span',first = False)[1].text,
                '收藏人數':html.find('div.count',first = False)[0].text,
                '提問人數':html.find('div.count',first = False)[1].text,
                '商品分類':html.find('div.breadcrumbs-set',first = True).text,
                '商品標籤':html.find('div.tags',first = True).text,
                '連結':'https://www.pcone.com.tw/product/info/' + re.search(r'\d{12}',str(info)).group()}],
            columns = ['店家名稱', '店家商品數量', '店家評價', '店家出貨天數', '店家回覆率',  '產品名稱', '特價', '原價', '折數',
                       '商品評分', '評價人數', '收藏人數','提問人數', '商品分類', '商品標籤', '連結']))

In [25]:
prodlist = ProdList(327)
df = pd.DataFrame()
for i in range(len(prodlist)):
    df = df.append(Crawl_SongGuo(prodlist[i]), ignore_index=True)

190117048588
180823036685
181212921322
180831227042
181011285850
190118229702
170809105334
190109230172
180918226398
190211403892
190406309250
190227225462
190403015320
190403299878
190403292961
190404996413
190405751750
190405612847
190406297782
190403505101
190406290891
190404570967
190403758109
190404091085
190404498869
190404833355
190403535217
190404163742
190405994645
190406577728
190404481875
190403747622
190404535099
190403953316
190403948691
190403465477
190404400552
190403261336
190404154482
190405243408
190405273864
190404297435
190227100853
190403554280
190403190463
190404579534
190402805449
190404429947
190404439309
190403924120


In [26]:
df

,店家名稱,店家商品數量,店家評價,店家出貨天數,店家回覆率,產品名稱,特價,原價,折數,商品評分,評價人數,收藏人數,提問人數,商品分類,商品標籤,連結
0,【好康市集】,28,4.6,0.9,94%,【瑞典】旅行折疊電熱水壺,$1280,$1699,7.5折,3.5,2,64,2,戶外休閒 旅行用品 旅用配件,#攜帶水壺 #電熱水壺 #旅行配件 #旅行水壺 #摺疊水壺 #收納水壺,https://www.pcone.com.tw/product/info/19011704...
1,草根王市集,76,4.3,0.7,99%,無印風U型連帽護頸枕,$349,$899,3.9折,3.8,32,596,10,戶外休閒 旅行用品 旅用配件,#連帽護頸枕,https://www.pcone.com.tw/product/info/18082303...
2,鵬博量販百貨,321,4.3,0.9,99%,德式304不鏽鋼面板抗菌砧板,$525,$1000,5.2折,1,0,77,,居家生活 餐飲廚具 刀具/砧板,#砧板 #切菜板 #擀麵板 #切菜 #防霉 #防鏽 #304不鏽鋼 #廚房 #酒店 #餐廳,https://www.pcone.com.tw/product/info/18121292...
3,【MUNA家居】,570,4.8,-,100%,【MUNA】凱爾 7 X 7尺積層色雙色衣櫥(另有白色),$9870,$14300,6.9折,5,2,77,12,寢具傢飾 家具 櫃子,#衣櫥 #櫥櫃 #櫃子 #衣櫃,https://www.pcone.com.tw/product/info/18083122...
4,愛屋家具,490,3.9,0.4,96%,IHouse-SGS 防潮抗蟲蛀塑鋼緩衝收納掃具櫃(寬62深47.5高190cm),$8940,$14890,6折,1,0,15,,寢具傢飾 家具 櫃子,#ihouse #塑鋼 #防水 #防潮 #無毒 #環保 #電器櫃 #置物櫃 #SGS認證 #吊櫃,https://www.pcone.com.tw/product/info/18101128...
5,中興,56,4.6,3.6,98%,飛樂 K10 全新健康免油氣炸鍋,$2980,$3480,8.6折,4.5,2,38,1,3C家電 生活家電 廚房家電,#氣炸鍋 #日本氣炸鍋 #arlink #EC-103 #飛樂氣炸鍋 #ARLINK氣炸鍋 ...,https://www.pcone.com.tw/product/info/19011822...
6,敏大企業,133,4.7,0.6,100%,"""獨家"" 德國BRITA 新版PLUS壺用濾芯 新春降價 買越多越划算 歡迎點進參考~",$170,$980,1.7折,4.7,42,328,23,居家生活 餐飲廚具 濾水/淨水,#BRITA #象印 #水壺 #濾水壺 #淨水器,https://www.pcone.com.tw/product/info/17080910...
7,生活百貨,355,4.1,1.1,97%,懶人舒適彈力伸縮隱形腰帶,$239,$599,4折,5,1,52,1,服飾婦幼 配件 其他服飾配件,#防盜皮帶 #牛皮皮帶 #牛皮 #自動皮帶 #自動皮帶頭 #真牛皮皮帶 #皮帶 #自動扣皮帶...,https://www.pcone.com.tw/product/info/19010923...
8,七號倉儲商行,6,4.8,0.7,100%,七號倉儲~就賀呷蛋捲經濟大包裝,$156,$280,5.6折,4.8,19,91,6,美食 零食/甜點 餅乾/零嘴,#手工蛋捲 #伴手禮 #零食 #下午茶 #點心 #原味 #芝麻 #黑糖 #咖啡,https://www.pcone.com.tw/product/info/18091822...
9,中興,56,4.6,3.6,98%,預購 Arlink 健康免油氣炸鍋 EC-103,$3780,$4480,8.4折,5,5,121,14,3C家電 生活家電 廚房家電,#arlink #氣炸鍋 #飛樂,https://www.pcone.com.tw/product/info/19021140...


In [27]:
df.to_excel('C:/Users/TLYu0419/Desktop/SongGuo.xlsx')

In [22]:
Crawl_SongGuo(190117048588)

NameError: name 'Crawl_SongGuo' is not defined

In [50]:
def ProdList(info):
    resp = requests.get(url + str(info), headers=headers)
    html = HTML(html=resp.text)
    prodlist = html.find('a.product-list-item')